In [ ]:
import calendar
import datetime
import json
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy.stats as spstats
from sqlalchemy import distinct, func, select, sql
import database
from database.schema import Rating, revision_table
import logbook

In [ ]:
exp_name = "24i_find_imp_controls"
exp = logbook.Experiment(exp_name)
project_file = "data/projects-2016-10-14-dedup.json"
out_file = "controls.csv"

In [ ]:
project_ids = []
with open(project_file, "rb") as f:
    for row in f:
        data = json.loads(row)
        project_ids.append(data["project_id"])

In [ ]:
conn = database.engine.connect()
try:
    timestamps = {}
    article_count = {}
    contributor_count = {}
    revision_count = {}
    article_age = {}
    mean_editors = {}
    most_recent_timestamp = None
    for project_id in project_ids:
        tab = revision_table(project_id).__table__
        # Revision count
        stmt = select([func.count(tab.c.internal_id)]) \
            .where(tab.c.important == True)
        result = conn.execute(stmt)
        count = result.fetchone()[0]
        if (count == 0):
            continue
        revision_count[project_id] = count
        # Article count
        stmt = select([func.count(distinct(tab.c.article_id))]) \
            .where(tab.c.important == True)        
        result = conn.execute(stmt)
        article_count[project_id] = result.fetchone()[0]
        # Total editor count
        stmt = select([func.count(distinct(tab.c.contributor_id))])
            .where(tab.c.important == True)
        result = conn.execute(stmt)
        contributor_count[project_id] = result.fetchone()[0]
        # Editors per article
        stmt = select([func.count(distinct(tab.c.contributor_id))]) \
            .group_by(tab.c.article_id) \
            .where(tab.c.important == True)
        result = conn.execute(stmt)
        count = 0
        total = 0
        for row in result:
            count += 1
            total += row[0]
        mean_editors[project_id] = float(total) / float(count)
        # Article age
        first_ts = func.min(tab.c.timestamp).label("first")
        subq = select([first_ts]) \
            .group_by(tab.c.article_id).alias('subq') \
            .where(tab.c.important == True)
        stmt = select([subq.c.first])
        result = conn.execute(stmt)
        article_age[project_id] = result.fetchone()[0]
        # Get most recent timestamp in project for age calcluation
        stmt = select([func.max(tab.c.timestamp)]) \
            .where(tab.c.important == True)
        result = conn.execute(stmt)
        ts = result.fetchone()[0]
        if most_recent_timestamp is None or ts > most_recent_timestamp:
            most_recent_timestamp = ts
        # Get earliest rating
        tab = Rating.__table__
        stmt = select([tab.c.timestamp]) \
            .where(
                sql.and_(
                    tab.c.project_id == project_id
                    tab.c.important == True)) \
            .order_by(tab.c.timestamp).limit(1)
        result = conn.execute(stmt)
        try:
            timestamps[project_id] = result.fetchone()[0]
        except TypeError:
            pass
    # Change datetime to age in seconds
    for project_id in project_ids:
        try:
            article_age[project_id] = most_recent_timestamp - article_age[project_id]
        except KeyError:
            pass
finally:
    conn.close()

In [ ]:
df_control = pd.DataFrame({
    'project_id':dict([(x,x) for x in project_ids]),
    'first_assessment':timestamps,
    'article_count':article_count,
    'contributor_count':contributor_count,
    'revision_count':revision_count,
    'mean_article_age':article_age,
    'mean_editors':mean_editors})
df_control = df_control.set_index('project_id')
df_control.to_csv(exp.get_filename(out_file))